In [2]:
#TDC="0xfe91" #broadcast address for for MDC-Motherboards
TDC="0xfe4c"  #broadcast address for TRB3 standard TDCs
# this is the key for broadcasting the commands to all TDCs of the same  kind, 
# which means also reaching all PASTTRECS connected to this TDCS at the same time


from trbnet import TrbNet 
import os
lib = '/trbnettools/trbnetd/libtrbnet.so'
host = os.getenv("DAQOPSERVER")

t = TrbNet(libtrbnet=lib, daqopserver=host)


spi_queue = 0
spi_mem = {}

def spi(TDC_str,CONN,CHIP, data_list, **kwargs):
  f = ""
  
  TDC = int(TDC_str,16)
  
  
  if ( not( TDC in spi_mem ) ):
    spi_mem[TDC] = {}
  if ( not( CONN in spi_mem[TDC] ) ):
    spi_mem[TDC][CONN] = {}
  if ( not( CHIP in spi_mem[TDC][CONN] ) ):
    spi_mem[TDC][CONN][CHIP] = []
  
  if spi_queue:
    
    spi_mem[TDC][CONN][CHIP] += data_list
  
  else:
 
    my_data_list = spi_mem[TDC][CONN][CHIP] + data_list
    spi_mem[TDC][CONN][CHIP].clear() # empty queue
 
    header = 0x52000
    if( CHIP == 1 ):
      header = 0x54000
      
    # bring all CS (reset lines) in the default state (1) - upper four nibbles: invert CS, lower four nibbles: disable CS
    t.trb_register_write(TDC, 0xd417, 0x0000FFFF)
   
    # (chip-)select output $CONN for i/o multiplexer reasons, remember CS lines are disabled
    t.trb_register_write(TDC, 0xd410, 0xFFFF & ( 1<<(CONN-1) ) )


    # disable all SDO outputs but output $CONN
    t.trb_register_write(TDC, 0xd415, 0xFFFF & ~(1<<(CONN-1)) )

    # disable all SCK outputs but output $CONN
    t.trb_register_write(TDC, 0xd416, 0xFFFF & ~(1<<(CONN-1)) )
 

    for data in my_data_list:
      # writing one data word, append zero to the data word, the chip will get some more SCK clock cycles
      t.trb_register_write(TDC, 0xd400, (header+data)<<4 )
    
      # write 1 to length register to trigger sending
      t.trb_register_write(TDC, 0xd411, 0x0001)
  
    
import time
for thr in range(0,12):
    time.sleep(1)
    thresh = thr*10
    # each TDC serves two PASTTREC boards (CONN = 1,2) with each having two PASTTRECS (CHIP = 0,1)
    for CONN in range(1,3): 
        for CHIP in range(0,2):
            spi(TDC,CONN,CHIP, [ 0x300 + (0xFF & thresh)])
            print(thresh, CONN, CHIP)


0 1 0
0 1 1
0 2 0
0 2 1
10 1 0
10 1 1
10 2 0
10 2 1
20 1 0
20 1 1
20 2 0
20 2 1
30 1 0
30 1 1
30 2 0
30 2 1
40 1 0
40 1 1
40 2 0
40 2 1
50 1 0
50 1 1
50 2 0
50 2 1
60 1 0
60 1 1
60 2 0
60 2 1
70 1 0
70 1 1
70 2 0
70 2 1
80 1 0
80 1 1
80 2 0
80 2 1
90 1 0
90 1 1
90 2 0
90 2 1
100 1 0
100 1 1
100 2 0
100 2 1
110 1 0
110 1 1
110 2 0
110 2 1
